<a href="https://colab.research.google.com/github/edgardominguez23/RandomWalk-Analysis/blob/main/TI_1_Practica4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 4

**Nombre:** Edgar Alan Dominguez Murillo  
**e-mail:** edgar.dominguez@alumnos.udg.mx

#MODULES

In [51]:
import math
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import panel as pn
from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

pn.extension(sizing_mode = 'stretch_width')

#CLASSES

In [52]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

#Actividad 4: Dashboard Random Walk + Path Length/MSD
* Funciones que generen trayectorias tipo Brownian Motion (BM), Correlated Random Walk (CRW) y Lévy Flight (LF).
* Cada una de las funciones deberá tomar como parámetros el numero de pasos, la velocidad y posición inicial. Además la funciones para CRW  y LF deberán tomar como parámetro el coeficiente para la distribución Cauchy. Por último, la función para LF también deberá aceptar como parámetro el exponente Lévy (alpha).

In [53]:
# Widgets
rw_type_select = pn.widgets.Select(options=['BM', 'CRW', 'LF'], name='Select an option')
n_steps_input = pn.widgets.IntInput(name='Number of steps',value=50,step=1,start=0,end=1000)
speed_input = pn.widgets.IntInput(name='Speed', value=6,step=1,start=0,end=10)
x_pos_input = pn.widgets.FloatInput(name='Starting pos x',value=0,step=1,start=0,end=10000)
y_pos_input = pn.widgets.FloatInput(name='starting pos y',value=0,step=1,start=0,end=10000)
exp_input = pn.widgets.FloatInput(name='Coefficient',value=0.6, step=0.1, start=0.1, end=1)
exp_levy_input = pn.widgets.FloatInput(name='Exponent levy',value=0.6, step=0.1, start=0.1, end=1)
m_type_select = pn.widgets.Select(options=['PL', 'MSD'], name='Metrics type')

In [54]:
# Algorithms
def generate_bm(n_steps, speed, x_pos, y_pos):
  # Init velocity vector
  velocity = Vec2d(speed, 0)

  positions = pd.DataFrame(columns=['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': x_pos, 'y_pos': y_pos}])
  positions = pd.concat([positions, temp_df], ignore_index=True)

  for i in range(n_steps - 1):
      turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
      velocity = velocity.rotated(turn_angle)

      temp_df = pd.DataFrame([{'x_pos': positions.x_pos[i] + velocity.x,
                                'y_pos': positions.y_pos[i] + velocity.y}])
      positions = pd.concat([positions, temp_df], ignore_index=True)

  return positions

def generate_crw(n_steps, speed, x_pos, y_pos, crw_exponent):
  # Init parameters
  time_per_step = 0.00001
  resolution = 200

  # Init velocity vector
  velocity = Vec2d(speed,0)
  initial_positions = np.ones(shape=(n_steps,3))*0
  initial_positions[0,0] = x_pos
  initial_positions[0,1] = y_pos

  aux_domain = np.linspace(0, 2*np.pi, resolution)
  wrapcauchy_pdf = np.array([wrapcauchy.pdf(i, crw_exponent) for i in aux_domain])
  wrapcauchy_pdf /= np.sum(wrapcauchy_pdf)

  for i in range(1,n_steps):
    # Choose the angle of rotation
    turn_angle = np.random.choice(aux_domain, p=wrapcauchy_pdf)
    velocity = velocity.rotated(turn_angle)

    # Update location
    initial_positions[i,0] = initial_positions[i-1,0]+velocity.x
    initial_positions[i,1] = initial_positions[i-1,1]+velocity.y

    # Calculate distance traveled in this step
    distance_traveled = velocity.get_length() * time_per_step

    # Accumulate time
    initial_positions[i,2] = initial_positions[i-1,2]+distance_traveled

  return initial_positions

def generate_lf(n_steps, speed, x_pos, y_pos, crw_exponent, exp_levy):
  # Init parameters
  time_per_step = 0.0001
  max_step_length = 10.0
  std_motion_steps = 5.0

  positions = [{ 'x':x_pos, 'y':y_pos, 'z':0 }]

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  angles = levy_stable.rvs(exp_levy, crw_exponent, loc=std_motion_steps, size=n_steps-1)

  for i in range(n_steps-1):
    turn_angle = angles[i]

    # Generate a step length according to the Lévy distribution
    step_length = levy_stable.rvs(exp_levy, crw_exponent)
    step_length = min(step_length, max_step_length)

    velocity = velocity.rotated(turn_angle)

    # Calculate distance traveled in this step
    distance_traveled = velocity.get_length() * time_per_step * step_length

    new_position = {
        'x': positions[-1]['x'] + velocity.x * step_length,
        'y': positions[-1]['y'] + velocity.y * step_length,
        'z': positions[-1]['z'] + distance_traveled
    }

    positions.append(new_position)

  return pd.DataFrame(positions)

In [59]:
@pn.depends(rw_type=rw_type_select.param.value, n_steps=n_steps_input.param.value, speed=speed_input.param.value, x_pos=x_pos_input.param.value, y_pos=y_pos_input.param.value, exp=exp_input.param.value, exp_levy=exp_levy_input.param.value)
def update_panel1(rw_type, n_steps, speed, x_pos, y_pos, exp, exp_levy):
  if rw_type == "BM":
    trajectory = generate_bm(n_steps, speed, x_pos, y_pos)
    fig = go.Figure(data=[go.Scatter3d(x=trajectory.x_pos, y=trajectory.y_pos, z=trajectory.index, mode='lines')])
    fig.update_layout(
      scene=dict(
          xaxis=dict(title='x_pos (mm)'),
          yaxis=dict(title='y_pos (mm)'),
          zaxis=dict(title='time')
      ),
      title='BM trajectory in 3D'
    )

    return fig
  elif rw_type == "CRW":
    trajectory = generate_crw(n_steps, speed, x_pos, y_pos, exp)
    fig = go.Figure(data=[go.Scatter3d(x=trajectory[:,0], y=trajectory[:,1], z=trajectory[:,2], mode='lines')])
    fig.update_layout(
      scene=dict(
          xaxis=dict(title='x_pos (mm)'),
          yaxis=dict(title='y_pos (mm)'),
          zaxis=dict(title='time')
      ),
      title='CRW trajectory in 3D'
    )

    return fig
  else:
    trajectory = generate_lf(n_steps, speed, x_pos, y_pos, exp, exp_levy)
    fig = go.Figure(data=[go.Scatter3d(x=trajectory.x, y=trajectory.y, z=trajectory.z, mode='lines')])
    fig.update_layout(
      scene=dict(
          xaxis=dict(title='x_pos (mm)'),
          yaxis=dict(title='y_pos (mm)'),
          zaxis=dict(title='time')
      ),
      title='Levy Flight in 3D'
    )

    return fig

@pn.depends(rw_type=rw_type_select.param.value, n_steps=n_steps_input.param.value, speed=speed_input.param.value, x_pos=x_pos_input.param.value, y_pos=y_pos_input.param.value, exp=exp_input.param.value, exp_levy=exp_levy_input.param.value, m_type=m_type_select)
def update_panel2(rw_type, n_steps, speed, x_pos, y_pos, exp, exp_levy, m_type):
  if rw_type == "BM":
    if m_type == "PL":
      fig = go.Figure()
      trajectory = generate_bm(n_steps, speed, x_pos, y_pos)
      met_df = ([np.linalg.norm(trajectory.iloc[i-1] - trajectory.iloc[i]) for i in range(1, trajectory.shape[0])])
      pl_bm = np.cumsum(met_df)
      fig.add_trace(go.Scatter(
          x=np.arange(len(pl_bm)),
          y=pl_bm,
          marker = dict(size=2),
          mode = 'lines',
          name = 'Path Length',
          showlegend = True))
      fig.update_layout(title_text='Path Length')

      return fig
    else:
      msd_bm = np.empty(shape=(0))
      fig = go.Figure()
      trajectory = generate_bm(n_steps, speed, x_pos, y_pos)

      for tau in range(1, trajectory.shape[0]):
          met_df = ([(np.linalg.norm(trajectory.iloc[i-tau] - trajectory.iloc[i]))**2 for i in range(tau, trajectory.shape[0],1)])
          d = np.mean(met_df)
          msd_bm = np.append(msd_bm, d)
          msd_bm_df = pd.DataFrame()
          msd_bm_df['msd_bm'] = msd_bm

      fig.add_trace(go.Scatter(
          x=np.arange(len(msd_bm_df.index)),
          y=msd_bm_df.msd_bm,
          marker = dict(size=2),
          mode = 'lines',
          name = 'Mean Squared Displacement.',
          showlegend = True))
      return fig
  elif rw_type == "CRW":
    if m_type == "PL":
      fig = go.Figure()
      trajectory = generate_crw(n_steps, speed, x_pos, y_pos, exp)

      met_array = np.linalg.norm(trajectory[1:] - trajectory[:-1], axis=1)
      pl_crw = np.cumsum(met_array)

      fig.add_trace(go.Scatter(
          x=np.arange(len(pl_crw)),
          y=pl_crw,
          marker=dict(size=2),
          mode='lines',
          name='Path Length',
          showlegend=True))

      fig.update_layout(title_text='Path Length')

      return fig
    else:
      msd_crw = np.empty(shape=(0))
      fig = go.Figure()
      crw_array = generate_crw(n_steps, speed, x_pos, y_pos, exp)

      for tau in range(1, crw_array.shape[0]):
          met_array = np.empty(shape=(0))
          for i in range(tau, crw_array.shape[0]):
              met = np.linalg.norm(crw_array[i - tau] - crw_array[i])**2
              met_array = np.append(met_array, met)
          d = np.mean(met_array)
          msd_crw = np.append(msd_crw, d)

      fig.add_trace(go.Scatter(
          x=np.arange(len(msd_crw)),
          y=msd_crw,
          marker=dict(size=2),
          mode='lines',
          name='Mean Squared Displacement.',
          showlegend=True))

      return fig
  else:
    if m_type == "PL":
      fig = go.Figure()
      trajectory = generate_lf(n_steps, speed, x_pos, y_pos, exp, exp_levy)

      met_df = ([np.linalg.norm(trajectory.iloc[i-1]-trajectory.iloc[i]) for i in range(1, trajectory.shape[0])])
      pl_lv = np.cumsum(met_df)
      fig.add_trace(go.Scatter(
          x=np.arange(len(pl_lv)),
          y=pl_lv,
          marker = dict(size=2),
          mode = 'lines',
          name = 'Path Length',
          showlegend = True))

      fig.update_layout(title_text='Path Length')

      return fig
    else:
      msd_lv = np.empty(shape=(0))
      fig = go.Figure()
      trajectory = generate_lf(n_steps, speed, x_pos, y_pos, exp, exp_levy)

      for tau in range(1, trajectory.shape[0]):
          met_df = ([(np.linalg.norm(trajectory.iloc[i-tau]-trajectory.iloc[i]))**2 for i in range(tau, trajectory.shape[0],1)])
          d = np.mean(met_df)
          msd_lv = np.append(msd_lv, d)
          msd_lv_df = pd.DataFrame()
          msd_lv_df['msd_lvf'] = msd_lv

      fig.add_trace(go.Scatter(
          x=np.arange(len(msd_lv_df.index)),
          y=msd_lv_df.msd_lvf,
          marker = dict(size=2),
          mode = 'lines',
          name = 'Mean Squared Displacement.',
          showlegend = True))
      return fig

In [60]:
widget_panel = pn.WidgetBox(
    pn.widgets.StaticText(value='<b>RW Type</b>'),
    rw_type_select,
    pn.widgets.StaticText(value='<b>Parameters</b>'),
    n_steps_input,
    speed_input,
    x_pos_input,
    y_pos_input,
    exp_input,
    exp_levy_input,
    width=400,
)

graph_panel1 = pn.pane.Plotly(update_panel1)
graph_panel2 = pn.pane.Plotly(update_panel2)

app = pn.Row(
    widget_panel,
    pn.Row(
           pn.WidgetBox(
               graph_panel1
           ),
           pn.WidgetBox(
              m_type_select,
              graph_panel2
           )
    ),
)

app.servable()

Row(sizing_mode='stretch_width')
    [0] WidgetBox(width=400)
        [0] StaticText(sizing_mode='stretch_width', value='<b>RW Type</b>')
        [1] Select(options=['BM', 'CRW', 'LF'], sizing_mode='stretch_width', value='CRW')
        [2] StaticText(sizing_mode='stretch_width', value='<b>Parameters</b>')
        [3] IntInput(end=1000, name='Number of steps', sizing_mode='stretch_width', start=0, value=50)
        [4] IntInput(end=10, name='Speed', sizing_mode='stretch_width', start=0, value=6)
        [5] FloatInput(end=10000, name='Starting pos x', sizing_mode='stretch_width', start=0, step=1)
        [6] FloatInput(end=10000, name='starting pos y', sizing_mode='stretch_width', start=0, step=1)
        [7] FloatInput(end=1, name='Coefficient', sizing_mode='stretch_width', start=0.1, value=0.6)
        [8] FloatInput(end=1, name='Exponent levy', sizing_mode='stretch_width', start=0.1, value=0.6)
    [1] Row(sizing_mode='stretch_width')
        [0] WidgetBox(sizing_mode='stretch_width')
            [0] Plotly(Figure, sizing_mode='stretch_width')
        [1] WidgetBox(sizing_mode='stretch_width')
            [0] Select(name='Metrics type', options=['PL', 'MSD'], sizing_mode='stretch_width', value='MSD')
            [1] Plotly(Figure, sizing_mode='stretch_width')